# WiFi Kung Fu

A WiFi PCAP files **WiFi_traffic.pcap** and **Hidden_Network.pcap** are present in the current working directory. Perform the following tasks using these PCAP.

**Scapy** are installed on the machine.

## Tasks

**Task 1:** Import scapy and load all packets of WiFi_traffic.pcap into a list.

In [1]:
from scapy.all import *

packets = rdpcap('WiFi_traffic.pcap')

**Task 2:** Write a program to print the MAC address of all unique WiFi devices. Use packets from the loaded list (i.e. WiFi_traffic.pcap loaded into list in Task 1).

In [2]:
import sys
from scapy.all import *

devices = set()

def GetDevices(pkt) :
    if pkt.haslayer(Dot11) :
        if pkt.addr2 and ( pkt.addr2 not in devices ) :
            devices.add(pkt.addr2)
            print len(devices), pkt.addr2


for pkt in packets:
    GetDevices(pkt)


1 6c:19:8f:5f:81:74
2 c0:ee:fb:30:1f:6d
3 34:e6:ad:56:e1:04
4 5c:51:88:31:a0:3b
5 bc:ae:c5:c3:5e:01
6 c4:e9:84:09:7f:d3
7 c8:be:19:79:31:90
8 e4:f8:9c:96:a1:a9
9 c4:12:f5:bf:de:54
10 e8:de:27:16:87:18
11 40:e2:30:12:10:8a
12 10:08:b1:8c:f9:97
13 ec:22:80:c3:4a:68
14 3c:1e:04:28:0b:d7
15 bc:f6:85:4d:b2:93
16 54:b8:0a:58:0b:34
17 a4:d1:8c:c5:b9:3e
18 e8:de:27:16:70:c9
19 94:44:52:74:e6:7a
20 e0:06:e6:07:18:e5
21 f4:dc:f9:52:3e:0d
22 4c:80:93:94:d8:92
23 00:1e:40:ed:4b:0f
24 30:0d:43:13:3b:f0
25 fc:b0:c4:91:71:e2
26 fc:b0:c4:91:71:e1
27 f4:d6:fb:a1:47:d2
28 6c:72:20:6b:e3:ad
29 fc:b0:c4:91:71:e3


**Task 3:** Write a program to print the MAC address of all unique WiFi devices. Use packets from the loaded list (i.e. WiFi_traffic.pcap loaded into list in Task 1).

Please ignore the garbage values if you encounter any.

In [3]:
ssids = set()

def GetSSID(pkt) :

    if pkt.haslayer(Dot11Beacon) : 
        temp = pkt 

        while temp:
            temp = temp.getlayer(Dot11Elt) 
            if temp and temp.ID == 0 and (temp.info not in ssids) :	
                ssids.add(temp.info)
                print len(ssids), pkt.addr3, temp.info
                break 

            temp = temp.payload 

for pkt in packets:
    GetSSID(pkt)


1 6c:19:8f:5f:81:74 Home_Network
2 6c:19:8f:5f:81:74 Xz
3 6c:19:8f:5f:81:74 +R
4 6c:19:8f:5f:81:74 :�
5 6c:19:8f:5f:81:74 ��
6 bc:ae:c5:c3:5e:01 BinarySecuritySolutions
7 c8:be:19:79:31:90 Amazon Wood 
8 e8:de:27:16:87:18 SecurityTube_Open
9 ec:22:80:c3:4a:68 dg_patel
10 3c:1e:04:28:0b:d7 Angel Kusum
11 bc:f6:85:4d:b2:93 Nirmall
12 54:b8:0a:58:0b:34 GUNEEV
13 c4:12:f5:bf:de:54 Incredible Holidays
14 94:44:52:74:e6:7a belkin.367a
15 00:1e:40:ed:4b:0f Shubha
16 fc:b0:c4:91:71:e2 
17 3c:1e:04:28:0b:d7 F�
18 bc:ae:c5:c3:5e:01 ��
19 c4:12:f5:bf:de:54 N�
20 6c:72:20:6b:e3:ad Logistics


**Task 4:** Print the list of SSID requested in probe requests whenever a new probe request is encountered. Use packets from the loaded list (i.e. WiFi_traffic.pcap loaded into list in Task 1).

This way of printing is to mimic the behavior of a real time WiFi scanner which actively sniffs the WiFi traffic and prints the updated table everytime a new probe request is encountred.

In [4]:
clientprobes = set()

def GetClientProbes(pkt) :

    if pkt.haslayer(Dot11ProbeReq) :

        if len(pkt.info) > 0 : 
            testcase = pkt.addr2 + '---' + pkt.info 
            if testcase not in clientprobes :
                clientprobes.add(testcase) 
                print "New Probe Found:  " + pkt.addr2 + ' ' + pkt.info 

                print "\n------------Client Probes Table ---------------\n"
                counter = 1
                for probe in clientprobes :
                    [client, ssid] = probe.split('---')
                    print counter, client, ssid
                    counter = counter + 1 
                print "\n-----------------------------------------------\n"

for pkt in packets:
    GetClientProbes(pkt)


New Probe Found:  5c:51:88:31:a0:3b Home_Network

------------Client Probes Table ---------------

1 5c:51:88:31:a0:3b Home_Network

-----------------------------------------------

New Probe Found:  30:0d:43:13:3b:f0 LazyArtists

------------Client Probes Table ---------------

1 5c:51:88:31:a0:3b Home_Network
2 30:0d:43:13:3b:f0 LazyArtists

-----------------------------------------------

New Probe Found:  c4:e9:84:09:7f:d3 BinarySecuritySolutions

------------Client Probes Table ---------------

1 5c:51:88:31:a0:3b Home_Network
2 30:0d:43:13:3b:f0 LazyArtists
3 c4:e9:84:09:7f:d3 BinarySecuritySolutions

-----------------------------------------------



**Task 5:** Check if there is any hidden WiFi network present in the locaded list packets.

In [5]:
hidden_ssid_aps = set()

def FindHinddenSSID(pkt) :

    if pkt.haslayer(Dot11Beacon) :
        if not pkt.info :
            if pkt.addr3 not in hidden_ssid_aps :
                hidden_ssid_aps.add(pkt.addr3)
                print "HIDDEN SSID Network Found! BSSID: ", pkt.addr3
    elif pkt.haslayer(Dot11ProbeResp) and  ( pkt.addr3 in hidden_ssid_aps ) :
        print "HIDDEN SSID Uncovered! ", pkt.info, pkt.addr3 

for pkt in packets:
    FindHinddenSSID(pkt)

HIDDEN SSID Network Found! BSSID:  fc:b0:c4:91:71:e2
HIDDEN SSID Network Found! BSSID:  fc:b0:c4:91:71:e1
HIDDEN SSID Network Found! BSSID:  fc:b0:c4:91:71:e3


**Task 6:** There is a hidden network present in **Hidden_Network.pcap**. Write a program to uncover it i.e. find the SSID name for it.

In [1]:
from scapy.all import *

hidden_ssid_aps = set()

def FindHinddenSSID(pkt) :

    if pkt.haslayer(Dot11Beacon) :
        if not pkt.info :
            if pkt.addr3 not in hidden_ssid_aps :
                hidden_ssid_aps.add(pkt.addr3)
                print "HIDDEN SSID Network Found! BSSID: ", pkt.addr3
    elif pkt.haslayer(Dot11ProbeResp) and  ( pkt.addr3 in hidden_ssid_aps ) :
        print "HIDDEN SSID Uncovered! ", pkt.info, pkt.addr3 

packets = rdpcap('Hidden_Network.pcap')

for pkt in packets:
    FindHinddenSSID(pkt)

HIDDEN SSID Network Found! BSSID:  50:c7:bf:c8:2a:1b
HIDDEN SSID Uncovered!  AttackDefense 50:c7:bf:c8:2a:1b
HIDDEN SSID Uncovered!  AttackDefense 50:c7:bf:c8:2a:1b
HIDDEN SSID Uncovered!  AttackDefense 50:c7:bf:c8:2a:1b
HIDDEN SSID Uncovered!  AttackDefense 50:c7:bf:c8:2a:1b
HIDDEN SSID Uncovered!  AttackDefense 50:c7:bf:c8:2a:1b
HIDDEN SSID Uncovered!  AttackDefense 50:c7:bf:c8:2a:1b
